In [1]:
import os
%pwd
os.chdir('../')

In [2]:
%pwd

'c:\\Users\\lenovo\\Desktop\\New folder (2)\\CNN_classifier'

# Training 

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int 
    params_is_augmentation: bool
    params_image_size:list



@dataclass(frozen=True)
class PrepareCallnacksConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path

In [29]:
from CnnClassifier.constants import *
from CnnClassifier.utils import read_yaml,create_directories
import tensorflow as tf

In [40]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallnacksConfig:
        config=self.config.prepare_callbacks
        model_chkpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_chkpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        preapare_callback_config=PrepareCallnacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return preapare_callback_config 
    
    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config
        

In [41]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [69]:
class PrepareCallnacks:
    def __init__(self,config: PrepareCallnacksConfig):
    
        self.config=config

    @property
    def create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_chkpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_chkpt_callbacks(self):
        return [
            self.create_tb_callbacks,
            self._create_chkpt_callbacks
        ]
    

In [74]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config=config
    def get_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenrator_kwargs=dict(
            rescale= 1./255,
            validation_split=0.20
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenrator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagnerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenrator_kwargs
            )
        else:
            train_datagnerator=valid_datagenerator
            
        self.train_generator=train_datagnerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path:Path,model: tf.keras.Model):
            model.save(path)

    def train(self,callaback_list: list):
            self.steps_per_epochs=self.train_generator.samples//self.train_generator.batch_size
            self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size

            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epochs,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                callbacks=callaback_list
            )
            self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )

In [76]:
try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callbacks_config()
    prepare_callback=PrepareCallnacks(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_chkpt_callbacks()

    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callaback_list=callback_list
    )
except  Exception as e:
    raise e

[2023-06-19 21:30:41,463: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-06-19 21:30:41,471: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-06-19 21:30:41,471: INFO: common: create directory  at: artifacts]
[2023-06-19 21:30:41,471: INFO: common: create directory  at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-19 21:30:41,479: INFO: common: create directory  at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-06-19 21:30:41,487: INFO: common: create directory  at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.
19/19 [==============================] - 194s 10s/step - loss: 12.7766 - accuracy: 0.5526 - val_loss: 10.1221 - val_accuracy: 0.3906
